# Retrieval-augmented Thoughts

In [1]:
# source: https://medium.com/@greyboi/ddgsearch-search-duckduckgo-scrape-the-results-in-python-18f5265f1aa6
%pip install --upgrade --quiet duckduckgo-search

Note: you may need to restart the kernel to use updated packages.


In [2]:
# %%capture
# !pip install boto3 --upgrade

In [3]:
import boto3
import os
import json
import re

bedrock = boto3.client(service_name='bedrock-runtime')

In [4]:
# model params
max_tokens = 4096
temperature = 0
top_k = 250 # optional
top_p = 0.999 # optional

anthropic_version = 'bedrock-2023-05-31'
accept = 'application/json'
contentType = 'application/json'

# set the Claude 3  model ID in Bedrock
model_id = 'anthropic.claude-3-sonnet-20240229-v1:0'

In [50]:
# Basic Tool Functions
import os
import json

from duckduckgo_search import DDGS
def get_search(query:str="", k:int=3): # get the top-k resources with DuckDuckGo
    return DDGS().text(query.replace('"',''), max_results=k)

from langchain_community.document_transformers import Html2TextTransformer
from langchain_community.document_loaders import AsyncHtmlLoader
def get_page_content(href:str):
    loader = AsyncHtmlLoader([href])
    docs = loader.load()
    html2text = Html2TextTransformer()
    docs_transformed = html2text.transform_documents(docs)
    if len(docs_transformed) > 0:
        return docs_transformed[0].page_content
    else:
        return None

import tiktoken
def num_tokens_from_string(string: str, encoding_name: str = "cl100k_base") -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

def chunk_text_by_sentence(text, chunk_size=2048):
    """Chunk the $text into sentences with less than 2k tokens."""
    sentences = text.split('. ')
    chunked_text = []
    curr_chunk = []
    # 逐句添加文本片段，确保每个段落都小于2k个token
    for sentence in sentences:
        if num_tokens_from_string(". ".join(curr_chunk)) + num_tokens_from_string(sentence) + 2 <= chunk_size:
            curr_chunk.append(sentence)
        else:
            chunked_text.append(". ".join(curr_chunk))
            curr_chunk = [sentence]
    # 添加最后一个片段
    if curr_chunk:
        chunked_text.append(". ".join(curr_chunk))
    return chunked_text[0]

def chunk_text_front(text, chunk_size = 2048):
    '''
    get the first `trunk_size` token of text
    '''
    chunked_text = ""
    tokens = num_tokens_from_string(text)
    if tokens < chunk_size:
        return text
    else:
        ratio = float(chunk_size) / tokens
        char_num = int(len(text) * ratio)
        return text[:char_num]

def chunk_texts(text, chunk_size = 2048):
    '''
    trunk the text into n parts, return a list of text
    [text, text, text]
    '''
    tokens = num_tokens_from_string(text)
    if tokens < chunk_size:
        return [text]
    else:
        texts = []
        n = int(tokens/chunk_size) + 1
        # 计算每个部分的长度
        part_length = len(text) // n
        # 如果不能整除，则最后一个部分会包含额外的字符
        extra = len(text) % n
        parts = []
        start = 0

        for i in range(n):
            # 对于前extra个部分，每个部分多分配一个字符
            end = start + part_length + (1 if i < extra else 0)
            parts.append(text[start:end])
            start = end
        return parts

In [74]:
# RAT Pipeline

from datetime import datetime
anthropic_system_prompt = f'''
You are an AI assistant called Claude, created by Anthropic. The current date is {datetime.now().strftime('%Y-%m-%d')}.
Claude's knowledge base was last updated on August 2023. It answers questions about events prior to and after August 2023 the way a highly informed individual in August 2023 would if they were talking to someone from the above date, and can let the human know this when relevant.
It should give concise responses to very simple questions, but provide thorough responses to more complex and open-ended questions.
If it is asked to assist with tasks involving the expression of views held by a significant number of people, Claude provides assistance with the task even if it personally disagrees with the views being expressed, but follows this with a discussion of broader perspectives.
Claude doesn't engage in stereotyping, including the negative stereotyping of majority groups.
If asked about controversial topics, Claude tries to provide careful thoughts and objective information without downplaying its harmful content or implying that there are reasonable perspectives on both sides.
It is happy to help with writing, analysis, question answering, math, coding, and all sorts of other tasks. It uses markdown for coding.
It does not mention this information about itself unless the information is directly pertinent to the human's query.
'''

def get_draft(question):
    # Getting the draft answer
    draft_prompt = '''
IMPORTANT:
Try to answer this question/instruction with step-by-step thoughts and make the answer more structured.
Use `\n\n` to split the answer into several paragraphs.
Just respond to the instruction directly. DO NOT add additional explanations or text in the answer unless you are asked to.
'''
    messages=[{"role": "user", "content": f"{question}" + draft_prompt}]

    body = json.dumps(
        {
            "max_tokens": max_tokens,
            "temperature": temperature,
            "top_k": top_k,
            "top_p": top_p,
            "anthropic_version": anthropic_version,
            "system": anthropic_system_prompt,
            "messages":  messages,
        })
    
    response = bedrock.invoke_model(body=body, modelId=model_id, accept=accept, contentType=contentType)
    response_body = json.loads(response.get('body').read())
    return response_body['content'][0]['text']

def split_draft(draft, split_char = '\n\n'):
    # 将draft切分为多个段落
    # split_char: '\n\n'
    draft_paragraphs = draft.split(split_char)
    # print(f"The draft answer has {len(draft_paragraphs)}")
    return draft_paragraphs

def get_query(question, answer):
    query_prompt = '''
I want to verify the content correctness of the given question, especially the last sentences.
Please summarize the content with the corresponding question.
This summarization will be used as a query to given to a search engine.
The query should be short but needs to be specific to make sure the search engine can find related knowledge or web pages.
You can also use search syntax to make the query short and clear enough for the search engine to find relevant language data.
Try to make the query as relevant as possible to the last few sentences in the content.
**IMPORTANT**
Just output the query directly. DO NOT add additional explanations or text in the answer unless you are asked to.
'''
    messages=[{"role": "user","content": f"##Question: {question}\n\n##Content: {answer}\n\n##Instruction: {query_prompt}"}]

    body = json.dumps(
        {
            "max_tokens": max_tokens,
            "temperature": temperature,
            "top_k": top_k,
            "top_p": top_p,
            "anthropic_version": anthropic_version,
            "system": anthropic_system_prompt,
            "messages":  messages,
        })

    response = bedrock.invoke_model(body=body, modelId=model_id, accept=accept, contentType=contentType)
    response_body = json.loads(response.get('body').read())
    return response_body['content'][0]['text']


def get_content(query):
    res = get_search(query, 1)
    if not res:
        print(">>> No good search result was found")
        return None
    search_results = res[0]
    href = search_results['href'] # title, snippet
    res = get_page_content(href)
    if not res:
        print(f">>> No content was found in {href}")
        return None
    retrieved_text = res
    trunked_texts = chunk_texts(retrieved_text, 1500)
    trunked_texts = [trunked_text.replace('\n', " ") for trunked_text in trunked_texts]
    if trunked_texts: print(f"Funcióno bien para query: {query}")
    return trunked_texts

def get_revise_answer(question, answer, content):
    revise_prompt = '''
I want to revise the answer according to retrieved related text of the question in WIKI pages.
You need to check whether the answer is correct.
If you find some errors in the answer, revise the answer to make it better.
If you find some necessary details are ignored, add it to make the answer more plausible according to the related text.
If you find the answer is right and do not need to add more details, just output the original answer directly.
**IMPORTANT**
Try to keep the structure (multiple paragraphs with its subtitles) in the revised answer and make it more structured to improve understanding.
Split the paragraphs with `\n\n` characters.
Just output the revised answer directly. DO NOT add additional explanations or text in the revised answer unless you are asked to.
'''
    messages=[{"role": "user", "content": f"##Existing Text in Wiki Web: {content}\n\n##Question: {question}\n\n##Answer: {answer}\n\n##Instruction: {revise_prompt}"}]

    body = json.dumps(
        {
            "max_tokens": max_tokens,
            "temperature": temperature,
            "top_k": top_k,
            "top_p": top_p,
            "anthropic_version": anthropic_version,
            "system": anthropic_system_prompt,
            "messages":  messages,
        })

    response = bedrock.invoke_model(body=body, modelId=model_id, accept=accept, contentType=contentType)
    response_body = json.loads(response.get('body').read())
    return response_body['content'][0]['text']

def get_reflect_answer(question, answer):
    reflect_prompt = '''
Give a title for the answer of the question.
And add a subtitle to each paragraph in the answer and output the final answer using markdown format. 
This will make the answer to this question look more structured for better understanding.
**IMPORTANT**
Try to keep the structure (multiple paragraphs with its subtitles) in the response and make it more structured for understanding.
Split the paragraphs with \n\n characters.
Just output the revised answer directly. DO NOT add additional explanations or text in the revised answer unless you are asked to.
'''
    messages=[{"role": "user", "content": f"##Question:\n{question}\n\n##Answer:\n{answer}\n\n##Instruction:\n{reflect_prompt}"}]

    body = json.dumps(
        {
            "max_tokens": max_tokens,
            "temperature": temperature,
            "top_k": top_k,
            "top_p": top_p,
            "anthropic_version": anthropic_version,
            "system": anthropic_system_prompt,
            "messages":  messages,
        })

    response = bedrock.invoke_model(body=body, modelId=model_id, accept=accept, contentType=contentType)
    response_body = json.loads(response.get('body').read())
    return response_body['content'][0]['text']

In [52]:
query="ricardo fort empresario negocios inmobiliarios productos belleza"

In [75]:
def get_query_wrapper(q, question, answer):
    result = get_query(question, answer)
    q.put(result)  # pone el resultado en la cola

def get_content_wrapper(q, query):
    result = get_content(query)
    print(result)
    q.put(result)  # pone el resultado en la cola

def get_revise_answer_wrapper(q, question, answer, content):
    result = get_revise_answer(question, answer, content)
    q.put(result)

def get_reflect_answer_wrapper(q, question, answer):
    result = get_reflect_answer(question, answer)
    q.put(result)
    
# from multiprocessing import Process, Queue
# def run_with_timeout(func, timeout, *args, **kwargs):
#     q = Queue()  # 创建一个Queue对象用于进程间通信
#     # 创建一个进程来执行传入的函数，将Queue和其他*args、**kwargs作为参数传递
#     p = Process(target=func, args=(q, *args), kwargs=kwargs)
#     p.start()
#     # 等待进程完成或超时
#     p.join(timeout)
#     if p.is_alive():
#         print(f"{datetime.now()} [INFO] 函数{str(func)}执行已超时({timeout}s)，正在终止进程...")
#         p.terminate()  # 终止进程
#         p.join()  # 确保进程已经终止
#         result = None  # 超时情况下，我们没有结果
#     else:
#         print(f"{datetime.now()} [INFO] 函数{str(func)}执行成功完成")
#         result = q.get()  # 从队列中获取结果
#     return result

from multiprocessing import Process, Queue
from datetime import datetime

def run_with_timeout(func, timeout, *args, **kwargs):
    q = Queue()  # crea un objeto Queue para la comunicación entre procesos
    p = Process(target=func, args=(q, *args), kwargs=kwargs)
    p.start()
    p.join(timeout)
    if p.is_alive():
        # Log con timestamp más claro y explicación del timeout
        print(f"{datetime.now()} - [INFO] Timeout: La función {func.__name__} excedió el tiempo de {timeout}s.")
        p.terminate()
        p.join()
        return {'error': 'timeout', 'detail': f'Function {func.__name__} exceeded {timeout} seconds.'}
    else:
        # Log de éxito
        print(f"{datetime.now()} - [INFO] La función {func.__name__} completó su ejecución exitosamente.")
        if not q.empty():
            return q.get()
        else:
            # Manejo en caso de que la cola esté vacía (indicaría otro tipo de problema)
            return {'error': 'empty_queue', 'detail': 'No result was returned from the process.'}

In [69]:
from difflib import unified_diff
from IPython.display import display, HTML

def generate_diff_html(text1, text2):
    diff = unified_diff(text1.splitlines(keepends=True),
                        text2.splitlines(keepends=True),
                        fromfile='text1', tofile='text2')

    diff_html = ""
    for line in diff:
        if line.startswith('+'):
            diff_html += f"<div style='color:green;'>{line.rstrip()}</div>"
        elif line.startswith('-'):
            diff_html += f"<div style='color:red;'>{line.rstrip()}</div>"
        elif line.startswith('@'):
            diff_html += f"<div style='color:blue;'>{line.rstrip()}</div>"
        else:
            diff_html += f"{line.rstrip()}<br>"
    return diff_html

In [88]:
# RAT Function
newline_char = '\n'

def rat(question):
    print(f"{datetime.now()} [INFO] Te estoy armando un borrador con Claude3 ...")
    draft = get_draft(question)
    print(f"--------------------- Borrador ---------------------")
    print(draft)
    print(f"---------------------  Fin  ------------------------")

    print(f"{datetime.now()} [INFO] Procesando el borrador ...")
    draft_paragraphs = split_draft(draft)
    print(f"{datetime.now()} [INFO] Te lo dividí en {len(draft_paragraphs)} partes.")
    answer = ""
    for i, p in enumerate(draft_paragraphs):
        print(str("-")*80)
        print(f"{datetime.now()} [INFO] Vamos a revisar la parte [{i+1}/{len(draft_paragraphs)}] ...")
        answer = answer + '\n\n' + p
        print(f"[{i}/{len(draft_paragraphs)}] Respuesta original:\n{answer.replace(newline_char, ' ')}")

        # query = get_query(question, answer)
        print(f"{datetime.now()} [INFO] Generamos una 'query` que vamos a usar para buscar después ...")
        res = run_with_timeout(get_query_wrapper, 30, question, answer)
        if not res:
            print(f"{datetime.now()} [INFO] Hay un timeout, vamos a saltearlo ...")
            continue
        else:
            query = res
        print(f">>> {i}/{len(draft_paragraphs)} Query: {query.replace(newline_char, ' ')}")

        print(f"{datetime.now()} [INFO] Arrancamos a crawlear las páginas ...")
        # content = get_content(query)
        res = run_with_timeout(get_content_wrapper, 30, query)
        if not res:
            print(f"{datetime.now()} [INFO] Hay un timeout, vamos a saltearlo ...")
            continue
        else:
            content = res

        LIMIT = 2
        for j, c in enumerate(content):
            if  j >= LIMIT: # limit large number of network pages
                break
            print(f"{datetime.now()} [INFO] Revisamos las respuestas contra las páginas ...[{j}/{min(len(content),LIMIT)}]")
            # answer = get_revise_answer(question, answer, c)
            res = run_with_timeout(get_revise_answer_wrapper, 30, question, answer, c)
            if not res:
                print(f"{datetime.now()} [INFO] Hay un timeout, vamos a saltearlo ...")
                continue
            else:
                diff_html = generate_diff_html(answer, res)
                display(HTML(diff_html))
                answer = res
            print(f"{datetime.now()} [INFO] Respuesta revisada: [{j}/{min(len(content),3)}]")
        print(f"[{i}/{len(draft_paragraphs)}] RESPUESTA REVISADA:\n {answer.replace(newline_char, ' ')}")
    res = run_with_timeout(get_reflect_answer_wrapper, 30, question, answer)
    if not res:
        print(f"{datetime.now()} [INFO] Hay un timeout, vamos a saltearlo ...")
    else:
        answer = res
    return draft, answer

In [ ]:
draft, answer = rat("Contame de la vida de Ricardo Fort")

2024-04-11 17:44:24.659440 [INFO] Te estoy armando un borrador con Claude3 ...
--------------------- Borrador ---------------------
Aquí están mis pensamientos paso a paso sobre la vida de Ricardo Fort:

`Ricardo Fort fue un empresario, productor de televisión y celebridad argentina muy conocida. Nació en 1968 en Buenos Aires, Argentina.

Fort se hizo famoso por su participación en varios reality shows y programas de televisión, donde mostraba su extravagante estilo de vida y personalidad excéntrica. Fue conocido por su amor por las cirugías plásticas y su obsesión por mantener una apariencia joven.

A principios de la década de 2000, Fort fundó su propia productora de televisión y produjo varios programas exitosos. También incursionó en los negocios inmobiliarios y la venta de productos de belleza.

Lamentablemente, Ricardo Fort falleció en 2013 a la edad de 45 años debido a una insuficiencia cardíaca y otras complicaciones de salud. Dejó atrás a sus dos hijos gemelos y un legado como

2024-04-11 17:45:17.378051 [INFO] Respuesta revisada: [0/2]
2024-04-11 17:45:17.378108 [INFO] Revisamos las respuestas contra las páginas ...[1/2]
2024-04-11 17:45:25.580455 - [INFO] La función get_revise_answer_wrapper completó su ejecución exitosamente.


2024-04-11 17:45:25.583738 [INFO] Respuesta revisada: [1/2]
[0/5] RESPUESTA REVISADA:
 Biografía  Inicios y primeros años  Ricardo Fort nació el 23 de noviembre de 1968 en Venado Tuerto, provincia de Santa Fe, Argentina. Desde muy joven, se involucró en los negocios familiares dedicados a la venta de artículos de limpieza.  Carrera empresarial y fama mediática  En la década de 1990, Fort se mudó a Buenos Aires y fundó su propia empresa de limpieza llamada "Ricardo Fort S.A.", la cual se convirtió en un éxito comercial. A principios de la década de 2000, comenzó a aparecer en programas de televisión, donde se destacó por su personalidad extravagante y su estilo de vida ostentoso. Su fama creció rápidamente, convirtiéndose en un personaje polémico y controversial en el mundo del espectáculo argentino.  Vida personal y familia  Fort se casó en dos ocasiones, primero con Gustavo Martínez y luego con Marisa Bello. Tuvo dos hijos, Felipito y Martita, fruto de su relación con Marisa Bello. Su

2024-04-11 17:46:04.464796 [INFO] Respuesta revisada: [0/2]
2024-04-11 17:46:04.464848 [INFO] Revisamos las respuestas contra las páginas ...[1/2]
2024-04-11 17:46:16.423677 - [INFO] La función get_revise_answer_wrapper completó su ejecución exitosamente.


2024-04-11 17:46:16.427044 [INFO] Respuesta revisada: [1/2]
[1/5] RESPUESTA REVISADA:
 Biografía  Inicios y primeros años  Ricardo Fort nació el 5 de noviembre de 1968 en Venado Tuerto, provincia de Santa Fe, Argentina. Desde muy joven, se involucró en los negocios familiares dedicados a la venta de artículos de limpieza y productos de tocador.  Carrera empresarial y fama mediática  En la década de 1990, Fort se mudó a Buenos Aires y fundó su propia empresa de limpieza llamada "Ricardo Fort S.A.", la cual se convirtió en un éxito comercial. A principios de la década de 2000, comenzó a aparecer en programas de televisión, donde se destacó por su personalidad extravagante, su estilo de vida ostentoso y su gusto por las cirugías estéticas. Su fama creció rápidamente, convirtiéndose en un personaje polémico y controversial en el mundo del espectáculo argentino.  Vida personal y familia  Fort se casó en dos ocasiones, primero con Gustavo Martínez y luego con Marisa Bello. Tuvo dos hijos, Fe

...

2024-04-11 17:47:05.521873 [INFO] Respuesta revisada: [1/2]
[2/5] RESPUESTA REVISADA:
 Biografía  Inicios y primeros años  Ricardo Fort nació el 5 de noviembre de 1968 en Venado Tuerto, provincia de Santa Fe, Argentina. Desde muy joven, se involucró en los negocios familiares dedicados a la venta de artículos de limpieza y productos de tocador.  Carrera empresarial y fama mediática  En la década de 1990, Fort se mudó a Buenos Aires y fundó su propia empresa de limpieza llamada "Ricardo Fort S.A.", la cual se convirtió en un éxito comercial. A principios de la década de 2000, comenzó a aparecer en programas de televisión, donde se destacó por su personalidad extravagante, su estilo de vida ostentoso y su gusto por las cirugías estéticas. Su fama creció rápidamente, convirtiéndose en un personaje polémico y controversial en el mundo del espectáculo argentino. Participó en varios reality shows y programas de televisión, donde mostraba su extravagante estilo de vida y personalidad excéntri

2024-04-11 17:47:51.876463 [INFO] Respuesta revisada: [0/2]
2024-04-11 17:47:51.876513 [INFO] Revisamos las respuestas contra las páginas ...[1/2]
2024-04-11 17:48:04.003869 - [INFO] La función get_revise_answer_wrapper completó su ejecución exitosamente.


2024-04-11 17:48:04.007389 [INFO] Respuesta revisada: [1/2]
[3/5] RESPUESTA REVISADA:
 Biografía  Inicios y primeros años  Ricardo Fort nació el 5 de noviembre de 1968 en Venado Tuerto, provincia de Santa Fe, Argentina. Desde muy joven, se involucró en los negocios familiares dedicados a la venta de artículos de limpieza y productos de tocador.  Carrera empresarial y fama mediática  En la década de 1990, Fort se mudó a Buenos Aires y fundó su propia empresa de limpieza llamada "Ricardo Fort S.A.", la cual se convirtió en un éxito comercial. A principios de la década de 2000, comenzó a aparecer en programas de televisión, donde se destacó por su personalidad extravagante, su estilo de vida ostentoso y su gusto por las cirugías estéticas. Su fama creció rápidamente, convirtiéndose en un personaje polémico y controversial en el mundo del espectáculo argentino. Participó en varios reality shows y programas de televisión, donde mostraba su extravagante estilo de vida y personalidad excéntri

2024-04-11 17:48:45.844433 [INFO] Respuesta revisada: [0/2]
2024-04-11 17:48:45.844484 [INFO] Revisamos las respuestas contra las páginas ...[1/2]
2024-04-11 17:48:55.738995 - [INFO] La función get_revise_answer_wrapper completó su ejecución exitosamente.


2024-04-11 17:48:55.742298 [INFO] Respuesta revisada: [1/2]
[4/5] RESPUESTA REVISADA:
 Biografía  Inicios y primeros años  Ricardo Fort nació el 5 de noviembre de 1968 en Venado Tuerto, provincia de Santa Fe, Argentina. Desde muy joven, se involucró en los negocios familiares dedicados a la venta de artículos de limpieza y productos de tocador.  Carrera empresarial y fama mediática  En la década de 1990, Fort se mudó a Buenos Aires y fundó su propia empresa de limpieza llamada "Ricardo Fort S.A.", la cual se convirtió en un éxito comercial. A principios de la década de 2000, comenzó a aparecer en programas de televisión, donde se destacó por su personalidad extravagante, su estilo de vida ostentoso y su gusto por las cirugías estéticas. Su fama creció rápidamente, convirtiéndose en un personaje polémico y controversial en el mundo del espectáculo argentino. Participó en varios reality shows y programas de televisión, donde mostraba su extravagante estilo de vida y personalidad excéntri

In [12]:
diff_html = generate_diff_html(draft, answer)
display(HTML(diff_html))

---

In [68]:
run_with_timeout(get_content_wrapper, 5, query)

2024-04-11 17:26:11.956560 [INFO] 函数<function get_content_wrapper at 0x7fe5b43a1b40>执行已超时(5s)，正在终止进程...


In [57]:
get_content(query)

Fetching pages: 100%|##########| 1/1 [00:00<00:00, 10.71it/s]


Funcióno bien para query: ricardo fort empresario negocios inmobiliarios productos belleza


['Ir al contenido  Menú principal  Menú principal  mover a la barra lateral ocultar  Navegación    * Portada   * Portal de la comunidad   * Actualidad   * Cambios recientes   * Páginas nuevas   * Página aleatoria   * Ayuda   * Donaciones   * Notificar un error  Buscar  Buscar    * Crear una cuenta   * Acceder  Herramientas personales    * Crear una cuenta   * Acceder  Páginas para editores desconectados más información    * Contribuciones   * Discusión  ## Contenidos  mover a la barra lateral ocultar    * Inicio    * 1Primeros años    * 2Empresario  Alternar subsección Empresario      * 2.1Felfort      * 2.2Ranking de la revista Forbes    * 3Carrera artística  Alternar subsección Carrera artística      * 3.1Televisión      * 3.2Teatro    * 4Vida privada    * 5Filmografía  Alternar subsección Filmografía      * 5.1Televisión      * 5.2Teatro        * 5.2.1Obra teatral en Broadway basada en su vida    * 6Fallecimiento  Alternar subsección Fallecimiento      * 6.1Repercusiones        * 6.

In [ ]:
query="Hablame de Ricardo Fort, como se murió?"

In [ ]:
def get_search(query:str="", k:int=1): # get the top-k resources with DuckDuckGo
    return DDGS().text(query, max_results=k)

In [ ]:
format_string(a)

In [ ]:
get_content(res)

In [ ]:
res = get_search(a, 1)
if not res:
    print(">>> No good search result was found")
search_results = res[0]
href = search_results['href'] # title, snippet
res = get_page_content(href)
if not res:
    print(f">>> No content was found in {href}")
retrieved_text = res
trunked_texts = chunk_texts(retrieved_text, 1500)
trunked_texts = [trunked_text.replace('\n', " ") for trunked_text in trunked_texts]

In [ ]:
get_search(a)

In [ ]:
answer

In [ ]:
draft

In [ ]:
a = run_with_timeout(get_query_wrapper, 3, question, answer)

In [ ]:
a

In [ ]:
print(f"{datetime.now()} [INFO] Obteniendo borrador...")
draft = get_draft(question)
print(f"{datetime.now()} [INFO] Borrador devuelto")
print(f"##################### BORRADOR #######################")
print(draft)
print(f"#####################  FIN  #######################")

print(f"{datetime.now()} [INFO] Procesando borrador...")
draft_paragraphs = split_draft(draft)
print(f"{datetime.now()} [INFO] El borrador se dividió en {len(draft_paragraphs)} partes")
answer = ""
for i, p in enumerate(draft_paragraphs):
    print(str(i)*80)
    print(f"{datetime.now()} [INFO] Modificando parte {i+1}/{len(draft_paragraphs)}...")
    answer = answer + '\n\n' + p

    print(f"{datetime.now()} [INFO] Generando Query correspondiente...")
    res = run_with_timeout(get_query_wrapper, 3, question, answer)
    if not res:
        print(f"{datetime.now()} [INFO] Saltando pasos siguientes...")
        continue
    else:
        query = res
    print(f">>> {i}/{len(draft_paragraphs)} Query: {query.replace(newline_char, ' ')}")

    print(f"{datetime.now()} [INFO] Obteniendo contenido web...")
    res = run_with_timeout(get_content_wrapper, 5, query)
    if not res:
        print(f"{datetime.now()} [INFO] Saltando pasos siguientes...")
        continue
    else:
        content = res

    for j, c in enumerate(content):
        if j > 2:
            break
        print(f"{datetime.now()} [INFO] Modificando respuesta basada en el contenido web...[{j}/{min(len(content),3)}]")
        res = run_with_timeout(get_revise_answer_wrapper, 10, question, answer, c)
        if not res:
            print(f"{datetime.now()} [INFO] Saltando pasos siguientes... error en la respuesta")
            continue
        else:
            diff_html = generate_diff_html(answer, res)
            display(HTML(diff_html))
            answer = res
        print(f"{datetime.now()} [INFO] Modificación de respuesta completada[{j}/{min(len(content),3)}]")

In [ ]:
res

# Gradio Demo

In [ ]:
# %%capture
# !pip install gradio --upgrade

In [ ]:
def predict(message, history):
    # rat function?
    draft, answer = rat(message)
    return answer

In [ ]:
# Fast ChatBot demo
import gradio as gr
gr.ChatInterface(predict).launch(share=True, inline=False, debug=True)